Use PCA to transform part of num features (has no nan, inf feats). and use all components.

### Thing modified
1. PCA, naming cols and the cols_num, cols_feats
2. PCA in predict needs on train+test

### Import packages

In [ ]:
import time
import pickle
import itertools
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from utilities import cal_mape

from utilities import cal_score, convert_types, create_dt_feats

### Params

In [ ]:
is_log_target = True
is_per_area_target = True

map_target = {
    (True, True):   'log_per_area_total_price',
    (True, False):  'log_total_price',
    (False, True):  'per_area_total_price',
    (False, False): 'total_price'
}
col_target = map_target[(is_log_target, is_per_area_target)]

#cols_feats_add = ['encoded_latlon', 'building_per_land_area'] # add feats into model
cols_num_add = ['building_per_land_area']
cols_cat_add = ['encoded_latlon']

#cols_feats_rm = [] # TODO: implement before using it
#['day_in_year_txn_dt', 'day_in_year_building_complete_dt', 'dt_diff_building_complete_dt_txn_dt'] # dt num feats
#['dow_txn_dt', 'dow_building_complete_dt', 'month_txn_dt', 'month_building_complete_dt',
# 'quarter_txn_dt', 'quarter_building_complete_dt', 'year_txn_dt', 'year_building_complete_dt'] # dt cat feats

### Processing

In [ ]:
def pre_processing(df, cols_num, cols_cat): #, cols_feats
    # Convert types
    df[cols_num] = df[cols_num].astype('float32')
    
    # generate target
    if 'total_price' in df:
        df['total_price'] = df['total_price'].astype('float32')
        df['log_total_price'] = np.log1p(df['total_price'])
        df['per_area_total_price'] = df['total_price'] / df['building_area']
        df['log_per_area_total_price'] = np.log1p(df['per_area_total_price'])
        df['target'] = df[col_target]

    # generate lat/lon encoded
    kmeans = pickle.load(open('./object_kmeans_k6_combined.pkl', 'rb'))
    df['encoded_latlon'] = kmeans.predict(df[['lat','lon']])
    
    # generate dt feats
    df = create_dt_feats(df, 'txn_dt')
    df = create_dt_feats(df, 'building_complete_dt')
    df['dt_diff_building_complete_dt_txn_dt'] = df['txn_dt'] - df['building_complete_dt']
    
    # generate building to land ratio
    df['building_per_land_area'] = df['building_area'] / df['land_area']
 
    #cols_feats = cols_num + cols_cat + cols_feats_add
    
    return df  #, cols_feats

In [ ]:
from feature_engineering import CategoricalColumnsEncoder
from feature_engineering import TargetMeanEncoding
from feature_engineering import MultiLabelEncoding

class FeatureEngineering():
    def __init__(self, cols_num, cols_cat, cols_feats, col_target, n_ran_encode=4):
        # cols
        self.cols_num = cols_num
        self.cols_cat = cols_cat
        self.cols_feats = cols_feats
        self.col_target = col_target
        
        # Numbers
        self.n_ran_encode = n_ran_encode
        
        # encoders
        #self.tar_encoder = TargetMeanEncoding(col_target = self.col_target)
        #self.ran_encoder = MultiLabelEncoding(self.n_ran_encode)
        self.cat_astyper = CategoricalColumnsEncoder(mode='pandas')
        
    def fit_transform(self, df):
        df = df.copy()
    
#        encoded1 = self.tar_encoder.fit_transform(df[self.cols_cat], df[self.col_target])
#        encoded2 = self.ran_encoder.fit_transform(df[self.cols_cat])
        self.cat_astyper.fit_transform(df, self.cols_cat)
    
        return df
#        return pd.concat([df, encoded1, encoded2], axis=1)
    
    def transform(self, df):
        df = df.copy()
#        encoded1 = self.tar_encoder.transform(df[self.cols_cat])
#        encoded2 = self.ran_encoder.transform(df[self.cols_cat])     # TODO: add generated feats into cols_feats
        self.cat_astyper.transform(df)

        return df
#        return pd.concat([df, encoded1, encoded2], axis=1)


In [ ]:
def post_processing(y_pred, df):
    """Args: 
          y_pred (1d np.array): raw prediction y
          df (pd.Dataframe): input feature df
       Return: np.array
    """
    y_pred_final = y_pred.copy()
    
    if is_log_target:
        y_pred_final = np.expm1(y_pred_final)
        y_pred_final = np.clip(y_pred_final, 0, None)

    if is_per_area_target:
        y_pred_final = y_pred_final * df['building_area'].values
        
    return y_pred_final

### Columns

In [ ]:
cols_num = ['txn_dt', 'total_floor', 'building_complete_dt', 'parking_area', 'parking_price', 'txn_floor',
            'land_area', 'building_area', 'lat', 'lon', 'village_income_median', 'town_population', 'town_area',
            'town_population_density', 'doc_rate', 'master_rate', 'bachelor_rate', 'jobschool_rate',
            'highschool_rate', 'junior_rate', 'elementary_rate', 'born_rate', 'death_rate', 'marriage_rate',
            'divorce_rate', 'N_50', 'N_500', 'N_1000', 'N_5000', 'N_10000', 'I_10', 'I_50', 'I_100', 'I_250',
            'I_500', 'I_1000', 'I_5000', 'I_10000', 'I_MIN', 'II_10', 'II_50', 'II_100', 'II_250', 'II_500',
            'II_1000', 'II_5000', 'II_10000', 'II_MIN', 'III_10', 'III_50', 'III_100', 'III_250', 'III_500',
            'III_1000', 'III_5000', 'III_10000', 'III_MIN', 'IV_10', 'IV_50', 'IV_100', 'IV_250', 'IV_500',
            'IV_1000', 'IV_5000', 'IV_10000', 'IV_MIN', 'V_10', 'V_50', 'V_100', 'V_250', 'V_500', 'V_1000',
            'V_5000', 'V_10000', 'V_MIN', 'VI_50', 'VI_100', 'VI_250', 'VI_500', 'VI_1000', 'VI_5000', 'VI_10000',
            'VI_MIN', 'VII_10', 'VII_50', 'VII_100', 'VII_250', 'VII_500', 'VII_1000', 'VII_5000', 'VII_10000',
            'VII_MIN', 'VIII_10', 'VIII_50', 'VIII_100', 'VIII_250', 'VIII_500', 'VIII_1000', 'VIII_5000',
            'VIII_10000', 'VIII_MIN', 'IX_10', 'IX_50', 'IX_100', 'IX_250', 'IX_500', 'IX_1000', 'IX_5000',
            'IX_10000', 'IX_MIN', 'X_10', 'X_50', 'X_100', 'X_250', 'X_500', 'X_1000', 'X_5000', 'X_10000', 'X_MIN',
            'XI_10', 'XI_50', 'XI_100', 'XI_250', 'XI_500', 'XI_1000', 'XI_5000', 'XI_10000', 'XI_MIN', 'XII_10',
            'XII_50', 'XII_100', 'XII_250', 'XII_500', 'XII_1000', 'XII_5000', 'XII_10000', 'XII_MIN', 'XIII_10',
            'XIII_50', 'XIII_100', 'XIII_250', 'XIII_500', 'XIII_1000', 'XIII_5000', 'XIII_10000', 'XIII_MIN',
            'XIV_10', 'XIV_50', 'XIV_100', 'XIV_250', 'XIV_500', 'XIV_1000', 'XIV_5000', 'XIV_10000', 'XIV_MIN']
cols_cat = ['building_material', 'city', 'building_type', 'building_use', 'parking_way', 'town', 'village',
            'I_index_50', 'I_index_500', 'I_index_1000', 'II_index_50', 'II_index_500', 'II_index_1000',
            'III_index_50', 'III_index_500', 'III_index_1000', 'IV_index_50', 'IV_index_500', 'IV_index_1000',
            'IV_index_5000', 'V_index_50', 'V_index_500', 'V_index_1000', 'VI_10', 'VI_index_50', 'VI_index_500',
            'VI_index_1000', 'VII_index_50', 'VII_index_500', 'VII_index_1000', 'VIII_index_50', 'VIII_index_500',
            'VIII_index_1000', 'IX_index_50', 'IX_index_500', 'IX_index_1000', 'IX_index_5000', 'X_index_50',
            'X_index_500', 'X_index_1000', 'XI_index_50', 'XI_index_500', 'XI_index_1000', 'XI_index_5000',
            'XI_index_10000', 'XII_index_50', 'XII_index_500', 'XII_index_1000', 'XIII_index_50', 'XIII_index_500',
            'XIII_index_1000', 'XIII_index_5000', 'XIII_index_10000', 'XIV_index_50', 'XIV_index_500',
            'XIV_index_1000']
cols_feats = cols_num + cols_cat

# informational cols
cols_dt = ['txn_dt', 'building_complete_dt']
cols_bin = ['I_index_50', 'I_index_500', 'I_index_1000', 'II_index_50', 'II_index_500', 'II_index_1000',
            'III_index_50', 'III_index_500', 'III_index_1000', 'IV_index_50', 'IV_index_500', 'IV_index_1000',
            'IV_index_5000', 'V_index_50', 'V_index_500', 'V_index_1000', 'VI_10', 'VI_index_50', 'VI_index_500',
            'VI_index_1000', 'VII_index_50', 'VII_index_500', 'VII_index_1000', 'VIII_index_50', 'VIII_index_500',
            'VIII_index_1000', 'IX_index_50', 'IX_index_500', 'IX_index_1000', 'IX_index_5000', 'X_index_50',
            'X_index_500', 'X_index_1000', 'XI_index_50', 'XI_index_500', 'XI_index_1000', 'XI_index_5000',
            'XI_index_10000', 'XII_index_50', 'XII_index_500', 'XII_index_1000', 'XIII_index_50', 'XIII_index_500',
            'XIII_index_1000', 'XIII_index_5000', 'XIII_index_10000', 'XIV_index_50', 'XIV_index_500',
            'XIV_index_1000']

### Read data

In [ ]:
df = pd.read_csv('dataset/train.csv', dtype=object)
df.head(5)

### Preprocessing

In [ ]:
df = pre_processing(df, cols_num, cols_cat)

In [ ]:
cols_num = cols_num + [c for c in cols_num_add if c not in cols_num]
cols_cat = cols_cat + [c for c in cols_cat_add if c not in cols_cat]
cols_feats = cols_num + cols_cat

### PCA

In [ ]:
# choose cols
cols_pca= [c for c in cols_num if df[c].isnull().sum()==0]
cols_pca.remove('building_per_land_area')
print('Num feats used in pca:', len(cols_pca))

# do pca
pca_reducer = PCA(n_components=151)
df_pca = pca_reducer.fit_transform(df[cols_pca])
print(df_pca.shape)
df_pca = pd.DataFrame(df_pca)
df_pca.columns = [f'pca_{i}' for i in range(151)]
df = pd.concat([df, df_pca], axis=1)

In [ ]:
print(pca_reducer.n_components_)
print(pca_reducer.explained_variance_ratio_)

In [ ]:
cols_pca = cols_pca

cols_num = set(cols_num) - set(cols_pca) | set([f'pca_{i}' for i in range(151)])
cols_num = list(cols_num)
print(len(cols_num))
print(cols_num)

cols_feats = cols_num + cols_cat

### Grid search

In [ ]:
# grid search
params_fix = {'task': 'train',
              'boosting_type': 'gbdt',
              'objective': 'mse',
              'metric': 'mape',
              'learning_rate': 0.01 }

lgb_other_params = {'num_boost_round': 20000,
                    'verbose_eval': 2000,
                    'early_stopping_rounds': 1000 }

# prvs baseline best estimator
# ('objective', 'mse'), ('metric', 'mape'),
# ('learning_rate', 0.015), ('num_leaves', 255),
# ('feature_fraction', 0.5), ('min_data_in_leaf', 5),
# ('lambda_l1', 0.1), ('lambda_l2', 0.1)),
# 5524.863045737181,
# [5528.862755759428, 5530.862894274216, 5514.863487177898]]

#params_gsearch = {'max_depth':[-1, 8],
#                  'num_leaves': [200, 255]}

#params_gsearch = {'max_depth': [16],
#                  'num_leaves': [300],
#                  'feature_fraction': [0.5],
#                  'min_data_in_leaf': [10],
#                  'lambda_l1': [0.01],
#                  'lambda_l2': [0.1],
#                 }
# rounr 1
params_gsearch = {'max_depth': [-1, 8, 16, 24],
                  'num_leaves': [200, 255, 300],           # may reduce in dim-reduction exp
                  'feature_fraction': [0.5, 0.75, 1.0],
                  'min_data_in_leaf': [5, 10, 20],
                  'lambda_l1': [0, 0.01, 0.1],
                  'lambda_l2': [0, 0.01, 0.1],
                 }
# round 2
#params_gsearch = {'max_depth': [16, 20],
#                  'num_leaves': [275, 300, 350],           # may reduce in dim-reduction exp
#                  'feature_fraction': [0.2, 0.4, 0.5, 0.6],
#                  'min_data_in_leaf': [7, 10],
#                  'lambda_l1': [0, 0.01, 0.1],
#                  'lambda_l2': [0, 0.01, 0.1],
#                 }
N_fold = 3
N_iter = 100
score_low_bound = 5450 # if score smaller than this, skip the param set

#### Round 1

In [ ]:
# Round 1
rand_params_set = []
for i in range(N_iter):
    params_rand = {key: np.random.choice(params_gsearch[key]) for key in params_gsearch}
    while params_rand in rand_params_set:
        params_rand = {key: np.random.choice(params_gsearch[key]) for key in params_gsearch}
    rand_params_set.append(params_rand)
gsearch = {}

In [ ]:
t0 = time.time()
gsearch = {}
folds = KFold(n_splits=N_fold, shuffle=True, random_state=123)
for i_fold, (itrain, ival) in enumerate(folds.split(df)): # kfold
    print('==== Fold', i_fold+1, '====')
    
    # split train, val
    df_train = df.loc[itrain]
    df_val = df.loc[ival]
    
    # feat eng
    feat_eng = FeatureEngineering(cols_num, cols_cat, cols_feats, col_target)
    df_train = feat_eng.fit_transform(df_train)
    df_val = feat_eng.transform(df_val)

    # Construct lgb dataset
    lgb_train = lgb.Dataset(df_train[cols_feats], df_train[col_target]).construct()
    lgb_val = lgb.Dataset(df_val[cols_feats], df_val[col_target], reference=lgb_train).construct()
        
    # random sample - grid search
    rand_params_next = []
    for params_rand in rand_params_set:
        params = params_fix.copy()
        params.update( params_rand )

        model = lgb.train(params, lgb_train, valid_sets=lgb_val, **lgb_other_params)

        y_pred = model.predict(df_val[cols_feats])
        mse = mean_squared_error(df_val[col_target], y_pred)
        mae = mean_absolute_error(df_val[col_target], y_pred)
        mape = cal_mape(df_val[col_target], y_pred)
        
        y_pred_final = post_processing(y_pred, df_val)
        score = cal_score(df_val['total_price'], y_pred_final)

        print('params:', params)
        print('score, mse, mae, mape: %.5f, %.5f, %.5f, %.5f.' % (score, mse, mae, mape))

        if score < score_low_bound:
            continue

        rand_params_next.append(params_rand)
        tuple_params = tuple(params.items())
        if tuple_params not in gsearch:
            gsearch[tuple_params] = [[], [], [], [], []]
#        gsearch[tuple_params][0].append(model)
        gsearch[tuple_params][0].append(score)
        gsearch[tuple_params][1].append(mse)
        gsearch[tuple_params][2].append(mae)
        gsearch[tuple_params][3].append(mape)
        
    rand_params_set = rand_params_next
# aggregate, sort gsearch results
gsearch_results = [[key, *[np.mean(val_i) for val_i in val], *val] for key, val in gsearch.items()]
gsearch_results.sort(key= lambda x: x[1], reverse=True)
t1 = time.time()

In [ ]:
print('Total computing time:', t1-t0)

In [ ]:
for row in gsearch_results:
    print(*row)

In [ ]:
metrics = ['score', 'mse', 'mae', 'mape']

In [ ]:
df_gsearch_results = []
for row in gsearch_results:
    result = dict(row[0])
    result['score'] = row[1]
    result['mse'] = row[2]
    result['mae'] = row[3]
    result['mape'] = row[4]
    result['best_iteration'] = np.mean([m.best_iteration for m in row[5]])
    df_gsearch_results.append(result)
df_gsearch_results = pd.DataFrame(df_gsearch_results)
df_gsearch_results = df_gsearch_results[list(params_gsearch.keys())+metrics+['best_iteration']]
display(df_gsearch_results)

In [ ]:
for name in metrics:
    plt.plot(df_gsearch_results[name])
    plt.xlabel('rank'); plt.ylabel(name); plt.show()

In [ ]:
for p in params_gsearch.keys():
    plt.scatter(df_gsearch_results[p], df_gsearch_results['score'], s=3)
    plt.scatter(df_gsearch_results[p][:10], df_gsearch_results['score'][:10], s=3)
    plt.xlabel(p); plt.ylabel('score'); plt.show()

In [ ]:
for p in params_gsearch.keys():
    plt.scatter(df_gsearch_results[p], df_gsearch_results['mae'], s=3)
    plt.scatter(df_gsearch_results[p][:10], df_gsearch_results['mae'][:10], s=3)
    plt.xlabel(p); plt.ylabel('mae'); plt.show()

In [ ]:
df_gsearch_results.columns

In [ ]:
# best iteration
for c in df_gsearch_results.columns:
    if c != 'best_iteration':
        plt.scatter(df_gsearch_results['best_iteration'], df_gsearch_results[c])
        plt.ylabel(c)
    else:
        plt.hist(df_gsearch_results['best_iteration'], bins=30)
        plt.ylabel('N')        
    plt.xlabel('best_iteration')
    plt.show()

### Feature importance

In [ ]:
lgb.plot_importance(gsearch_results[0][5][0], figsize=(12, 30), height=0.5) # 050, 051, 052
plt.show()

In [ ]:
lgb.plot_importance(gsearch_results[0][5][0], figsize=(12, 30), height=0.5, importance_type='gain') # 050, 051, 052
plt.show()

Results:

(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 300), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.1), ('lambda_l2', 0.0)) 5562.8626109379065 0.04307499162341705 0.13229782226230116 0.009940239813132181 nan [5544.862518157318, 5568.864484355969, 5574.860830300434] [0.042241107856307275, 0.04362114598810233, 0.04336272102584154] [0.13198388317748455, 0.13213920963048423, 0.13277037397893476] [0.00992134541893296, 0.009911227014286343, 0.00998814700617724] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.1), ('lambda_l2', 0.1)) 5557.1963348811305 0.04243127521276202 0.13188663964017663 0.00990819958367564 nan [5542.862795704669, 5574.864784951336, 5553.861423987384] [0.04178030898761893, 0.04302518293928471, 0.04248833371138243] [0.1316918515637957, 0.1318287979129188, 0.13213926944381535] [0.00989811519294336, 0.009886989940738037, 0.009939493617345523] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.01), ('lambda_l2', 0.0)) 5556.529717962301 0.042814588627571586 0.13190488064126046 0.009909831348659458 nan [5532.862901257803, 5582.864918952132, 5553.86133367697] [0.04199522299794792, 0.04347451042619625, 0.04297403245857058] [0.1316203671247564, 0.1317820308258658, 0.13231224397315922] [0.009893276665108086, 0.009882748496088615, 0.009953468884781673] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.0), ('lambda_l2', 0.1)) 5555.196130349552 0.04282385061632852 0.13209310689960074 0.009923392850796272 nan [5535.86271879262, 5560.8646428334, 5568.8610294226355] [0.042023474207199464, 0.04344657893950197, 0.04300149870228413] [0.13182587650801506, 0.13195818354920813, 0.13249526064157902] [0.009907468711654087, 0.009896971639981702, 0.009965738200753029] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 300), ('feature_fraction', 0.5), ('min_data_in_leaf', 20), ('lambda_l1', 0.1), ('lambda_l2', 0.01)) 5554.528944058249 0.04293076389994658 0.1324248263754709 0.009949073454402048 nan [5534.862190572895, 5577.864220137023, 5550.860421464829] [0.04216208283156799, 0.04353736483331086, 0.04309284403496088] [0.1323274120011049, 0.13211033314921455, 0.13283673397609322] [0.009946223986211231, 0.00990849862281531, 0.0099924977541796] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.1), ('lambda_l2', 0.0)) 5553.529655012248 0.042772261073702134 0.13191089140625586 0.009910079472577915 nan [5522.862726277126, 5574.864823226549, 5562.861415533072] [0.04203654757429364, 0.04334987863565268, 0.04293035701116007] [0.13177922468233502, 0.1317900961769765, 0.1321633533594561] [0.009905299354914614, 0.009883172530770451, 0.009941766532048675] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 300), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.01), ('lambda_l2', 0.01)) 5553.529241360047 0.043049819476755895 0.13235428059107443 0.009943488023302567 nan [5536.862558252093, 5578.864480188164, 5544.860685639884] [0.04233231812875444, 0.043541690022388, 0.043275450279125244] [0.13203013579492834, 0.1321643209082156, 0.13286838507007936] [0.009923298887396731, 0.009911894737004092, 0.009995270445506878] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 300), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.01), ('lambda_l2', 0.1)) 5550.862723155597 0.04315892637517327 0.13224490253613908 0.009935418179224808 nan [5524.862564989204, 5572.86455313911, 5554.861051338478] [0.042373547463593356, 0.04378469274830669, 0.04331853891361974] [0.13204742683323237, 0.13213759914555032, 0.13254968162963454] [0.009925163137460115, 0.009910312564276475, 0.00997077883593783] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 20), ('lambda_l1', 0.1), ('lambda_l2', 0.0)) 5548.8628648817685 0.0424150405638588 0.13197107001837743 0.009913994802838277 nan [5516.862566449301, 5562.864590565644, 5566.861437630359] [0.04176927744762741, 0.04313343924054603, 0.042342405003402965] [0.13191341057517822, 0.13184486017105695, 0.13215493930889716] [0.00991407557487085, 0.009887519892171551, 0.00994038894147243] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.1), ('lambda_l2', 0.01)) 5547.8629895444 0.04254806603159524 0.13192251702471666 0.009910393762861892 nan [5522.862760224755, 5546.864652076949, 5573.861556331498] [0.04190559427709304, 0.043153980784162355, 0.042584623033530326] [0.13173988641359302, 0.13196304612440105, 0.13206461853615592] [0.00990172894540823, 0.009896040372015292, 0.009933411971162154] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.01), ('lambda_l2', 0.0)) 5547.5298028603165 0.042587030544331626 0.1318048870600032 0.009902092362798369 nan [5523.8629392986695, 5566.864973941588, 5551.861495340689] [0.04180109756771451, 0.043170431486210026, 0.042789562579070343] [0.1316200386977393, 0.13163104306533138, 0.13216357941693896] [0.00989267660711291, 0.009873060229607168, 0.00994054025167503] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.0), ('lambda_l2', 0.1)) 5547.529537803367 0.04279242229258745 0.13204045804201306 0.0099190607434912 nan [5518.862591691888, 5570.864906534286, 5552.861115183927] [0.04215519124799078, 0.04327666282824911, 0.04294541280152245] [0.1319827996280373, 0.13173990224999016, 0.13239867224801166] [0.009919282492593445, 0.009879051156535465, 0.009958848581344687] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 20), ('lambda_l1', 0.01), ('lambda_l2', 0.1)) 5546.529105599074 0.0427893093580494 0.13232653602841124 0.009940788643826125 nan [5529.862520649764, 5546.864256602429, 5562.860539545029] [0.04192933059390983, 0.04340623841615126, 0.0430323590640871] [0.13203651276427195, 0.13218346029992842, 0.13275963502103333] [0.009922129920444075, 0.009913000432173798, 0.009987235578860507] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.0), ('lambda_l2', 0.0)) 5546.196111972131 0.04285421997627193 0.13214186786485332 0.009926998322820011 nan [5539.862832541889, 5543.864539919142, 5554.8609634553595] [0.04201457486043407, 0.04342797632882085, 0.04312010873956088] [0.13179185715207029, 0.1320785285542469, 0.13255521788824282] [0.009904823806479491, 0.009905489801146127, 0.009970681360834413] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.0), ('lambda_l2', 0.01)) 5545.196292683002 0.0428588484758585 0.13200216482537327 0.009917626528441983 nan [5518.8626766142825, 5564.864773112968, 5551.861428321756] [0.04211877078022073, 0.04344162810563835, 0.04301614654171645] [0.13191154220126872, 0.13182503804260462, 0.13226991423224646] [0.009915455792049775, 0.009887012659994, 0.009950411133282176] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.0), ('lambda_l2', 0.01)) 5544.196314990585 0.04265557797479666 0.131952637586602 0.009912687008589516 nan [5518.862656955492, 5569.864677378593, 5543.861610637669] [0.041893043446694415, 0.04335969086739098, 0.04271399961030459] [0.1317601345723801, 0.13194873142107416, 0.13214904676635175] [0.009903918920810008, 0.009895361783262882, 0.00993878032169566] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.01), ('lambda_l2', 0.0)) 5543.863018386066 0.04259293798071989 0.13191438183838852 0.009910292397458657 nan [5535.862652426053, 5554.864710349666, 5540.8616923824775] [0.041862069135996445, 0.04332330793284064, 0.042593436873322586] [0.13180311379049758, 0.1318823249247103, 0.13205770679995768] [0.00990713634213322, 0.009890840620203824, 0.009932900230038927] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 20), ('lambda_l1', 0.01), ('lambda_l2', 0.0)) 5543.529231015867 0.04276855092335735 0.1321975014001775 0.009932185354179937 nan [5524.862400704177, 5544.864349636643, 5560.860942706782] [0.0420226838587232, 0.04336291176703578, 0.042920057144313066] [0.1320459223734877, 0.1321164604735807, 0.13243012135346413] [0.009925753698684347, 0.009908337320713804, 0.009962465043141657] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.1), ('lambda_l2', 0.0)) 5542.196345363264 0.04254225547180009 0.13189136833328075 0.009908550286265324 nan [5512.862601343063, 5556.864833450199, 5556.861601296532] [0.04195269659155342, 0.04310965333011982, 0.04256441649372703] [0.13185428187089127, 0.1317906567602277, 0.13202916636872328] [0.00991097206790278, 0.009883000398355223, 0.00993167839253797] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.01), ('lambda_l2', 0.1)) 5539.529753996946 0.04255568151174197 0.13187916368748376 0.009907326787054636 nan [5510.8627656729195, 5550.864860371791, 5556.861635946126] [0.041841718082811244, 0.043236670533985125, 0.04258865591842956] [0.13176150860133992, 0.13184616220421402, 0.13202982025689738] [0.009903527824174278, 0.009887629961378727, 0.009930822575610902] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 20), ('lambda_l1', 0.01), ('lambda_l2', 0.1)) 5539.19587804469 0.04273784945644179 0.1322385694083054 0.009934726320103214 nan [5502.862404821738, 5552.864286240224, 5561.860943072107] [0.04199374898606461, 0.04343811876197808, 0.04278168062128268] [0.1320908423797628, 0.13220947990123957, 0.1324153859439138] [0.009927449347226676, 0.009915189338405598, 0.00996154027467737] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.01), ('lambda_l2', 0.1)) 5538.862996153232 0.04255707051414679 0.13192158015155875 0.00991080089835702 nan [5514.862694210759, 5559.864783969148, 5541.861510279789] [0.04194193729037725, 0.04321423798467711, 0.04251503626738601] [0.13185976747894343, 0.13179370500637133, 0.13211126796936146] [0.009910419456787473, 0.009884828987420494, 0.009937154250863094] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 300), ('feature_fraction', 0.5), ('min_data_in_leaf', 20), ('lambda_l1', 0.0), ('lambda_l2', 0.1)) 5538.862438821784 0.04291244141218949 0.13230980768865838 0.009939515894994122 nan [5532.862416808035, 5556.864322309361, 5526.8605773479585] [0.04211928547717929, 0.043554981597923266, 0.043063057161465926] [0.13216671747323966, 0.1321112805045622, 0.13265142508817332] [0.009932607759548365, 0.009907667983418068, 0.00997827194201593] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 20), ('lambda_l1', 0.0), ('lambda_l2', 0.01)) 5534.862691687192 0.042535839963536076 0.13208735031650015 0.009922850316204934 nan [5508.862588167918, 5550.864208418299, 5544.861278475357] [0.04177388411628804, 0.04330101615391062, 0.04253261962040957] [0.13185714442476382, 0.1322037810600994, 0.13220112546463725] [0.009909460143218482, 0.009914574295392308, 0.009944516510004013] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 300), ('feature_fraction', 0.5), ('min_data_in_leaf', 20), ('lambda_l1', 0.01), ('lambda_l2', 0.01)) 5534.195614969663 0.042980262460802277 0.13248543359568465 0.009953569019824512 nan [5516.8623028654765, 5534.863858254103, 5550.860683789411] [0.04217646951159271, 0.04371872773236303, 0.04304559013845108] [0.13224250465922038, 0.1325636198892007, 0.13265017623863282] [0.009939166258974634, 0.00994268610065085, 0.009978854699848053] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 255), ('feature_fraction', 0.75), ('min_data_in_leaf', 5), ('lambda_l1', 0.01), ('lambda_l2', 0.1)) 5533.528461051344 0.04350921024301957 0.13305503261375642 0.00999338902357492 nan [5524.8611659773915, 5538.863989621318, 5536.860227555322] [0.04303081426991178, 0.04392218274865029, 0.04357463371049665] [0.1332069221452558, 0.13274929828575824, 0.13320887741025522] [0.010010455418747928, 0.009953315242751299, 0.010016396409225532] []  
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.0), ('lambda_l2', 0.01)) 5533.196195245051 0.04258877796016022 0.1319741700962258 0.009914633329693466 nan [5522.862737403599, 5523.864486411558, 5552.861361919998] [0.04185033444728114, 0.04329078091816382, 0.04262521851503572] [0.13169596224197666, 0.1320680572356505, 0.13215849081105027] [0.009898650592820003, 0.009904898030018521, 0.009940351366241874] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 300), ('feature_fraction', 0.75), ('min_data_in_leaf', 5), ('lambda_l1', 0.1), ('lambda_l2', 0.01)) 5532.52839757245 0.043651192014465996 0.133148965720051 0.010001160201469576 nan [5510.8614939816935, 5547.863647232057, 5538.860051503601] [0.04290144607913579, 0.04419346859671637, 0.043858661367545834] [0.13300051544975477, 0.13297955334884282, 0.13346682836155546] [0.009994062507064446, 0.009973140706468245, 0.010036277390876036] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 20), ('lambda_l1', 0.01), ('lambda_l2', 0.0)) 5532.195977475043 0.042543140470152195 0.1321315919949024 0.009926402754721493 nan [5493.862450921616, 5525.864311599609, 5576.861169903904] [0.04176146952547913, 0.04326680748795009, 0.04260114439702738] [0.13199390868431746, 0.13213122548529888, 0.13226964181509085] [0.009920454856735483, 0.009908811415848501, 0.009949941991580492] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 300), ('feature_fraction', 0.75), ('min_data_in_leaf', 5), ('lambda_l1', 0.0), ('lambda_l2', 0.0)) 5532.194990955963 0.04378827382489047 0.13323272492213573 0.01000776182331157 nan [5528.861486712012, 5536.863533151748, 5530.85995300413] [0.042988815691147424, 0.04437445022712878, 0.0440015555563952] [0.13296501257021165, 0.1331838139758797, 0.13354934822031583] [0.00999150989354827, 0.009988629877302998, 0.010043145699083445] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.0), ('lambda_l2', 0.01)) 5529.196026580998 0.04258353715180124 0.13222404521812794 0.009931935470184691 nan [5506.862138904236, 5548.864518968945, 5531.86142186981] [0.04195770116750451, 0.043223687875358754, 0.042569222412540464] [0.13219552729897227, 0.13228065041040157, 0.13219595794501005] [0.009935806370258907, 0.009918971824447027, 0.009941028215848136] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 255), ('feature_fraction', 0.75), ('min_data_in_leaf', 5), ('lambda_l1', 0.1), ('lambda_l2', 0.1)) 5529.195058600261 0.04348615372660763 0.13310966506913896 0.009997407999370076 nan [5512.861377976804, 5554.863650062795, 5519.860147761186] [0.042770151800746815, 0.04402591261322317, 0.043662396765852894] [0.13306440611235154, 0.13286978765385624, 0.13339480144120913] [0.009999061422303519, 0.009963608566962497, 0.010029554008844213] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 255), ('feature_fraction', 0.75), ('min_data_in_leaf', 10), ('lambda_l1', 0.01), ('lambda_l2', 0.0)) 5526.86142481173 0.04367309961689072 0.13329816738356579 0.010011870768367006 nan [5522.861285003833, 5534.863229271238, 5522.859760160119] [0.042861000871465084, 0.044334032479741076, 0.04382426549946601] [0.13309406441881153, 0.13319705627394499, 0.13360338145794087] [0.01000167498642883, 0.009987179258499828, 0.010046758060172361] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 255), ('feature_fraction', 0.75), ('min_data_in_leaf', 10), ('lambda_l1', 0.01), ('lambda_l2', 0.0)) 5526.528234314871 0.0436170346539575 0.13318088504251635 0.010002818071063598 nan [5516.861251837282, 5541.863496407443, 5520.8599546998885] [0.042884531771911094, 0.04427416602263861, 0.043692406167322784] [0.13309879885220513, 0.13299446106453083, 0.1334493952108131] [0.010001595553314788, 0.009971999216974288, 0.010034859442901719] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 300), ('feature_fraction', 0.75), ('min_data_in_leaf', 10), ('lambda_l1', 0.0), ('lambda_l2', 0.1)) 5526.194666742261 0.04382966055376695 0.1334695298730633 0.010024449377844162 nan [5510.861378677834, 5518.863079790578, 5548.859541758372] [0.04297978868859895, 0.04454899702033916, 0.043960195952362766] [0.1330871339469848, 0.13358643870872533, 0.13373501696347972] [0.010000504527726745, 0.010016605046249278, 0.010056238559556457] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 300), ('feature_fraction', 0.75), ('min_data_in_leaf', 10), ('lambda_l1', 0.0), ('lambda_l2', 0.0)) 5524.194687744224 0.043842922784052475 0.13339368438387306 0.010018990355577307 nan [5516.861232586964, 5526.863078658672, 5528.859751987038] [0.04306813863181975, 0.044555188460002795, 0.043905441260334896] [0.13320733047058714, 0.13339934177485724, 0.1335743809061748] [0.010010367292243564, 0.010003127424607263, 0.010043476349881095] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.01), ('lambda_l2', 0.01)) 5523.862444334954 0.04259528602385023 0.1323636572150939 0.009942225154631952 nan [5486.862030310259, 5534.864118862792, 5549.861183831809] [0.041922921284803315, 0.043266136408094784, 0.0425968003786526] [0.13231924716220914, 0.13248049586719096, 0.1322912286158816] [0.00994398580908848, 0.009934748869549113, 0.009947940785258262] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.0), ('lambda_l2', 0.1)) 5523.1958893096535 0.04266032476076651 0.13235243209989067 0.009941271445041687 nan [5488.861909917958, 5535.864511293106, 5544.861246717897] [0.04208955627547623, 0.043279796716128825, 0.04261162129069447] [0.13250417397194184, 0.13224634537167007, 0.13230677695606005] [0.009958846664922536, 0.009916346049607493, 0.009948621620595035] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.0), ('lambda_l2', 0.0)) 5522.52907273485 0.04263539252251717 0.13243490109562392 0.009946787699525913 nan [5490.861951642862, 5530.8641577006965, 5545.861108860992] [0.042016227712067554, 0.043253659935761295, 0.04263628991972267] [0.13241136513294322, 0.13246742930704705, 0.1324259088468815] [0.009950878304588224, 0.009931641165761859, 0.009957843628227655] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 300), ('feature_fraction', 0.75), ('min_data_in_leaf', 20), ('lambda_l1', 0.01), ('lambda_l2', 0.1)) 5521.861136910251 0.043733590915894766 0.13348948178002565 0.010026282300943016 nan [5488.860997979379, 5545.8629452440055, 5530.85946750737] [0.042823673356767344, 0.04446607469535274, 0.04391102469556421] [0.1333896119208428, 0.1333985272098539, 0.1336803062093802] [0.010021678539750473, 0.010003788385881526, 0.010053379977197047] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.0), ('lambda_l2', 0.1)) 5520.862390757019 0.042574632166520486 0.1324160793620303 0.009946524227883493 nan [5501.862031678588, 5527.864057935686, 5532.861082656783] [0.04187801974567095, 0.04331453963069181, 0.042531337123198676] [0.1322876991022972, 0.13257307135770954, 0.13238746762608408] [0.009942361183340694, 0.009941020635409806, 0.009956190864899978] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 200), ('feature_fraction', 0.75), ('min_data_in_leaf', 5), ('lambda_l1', 0.0), ('lambda_l2', 0.1)) 5519.861695932137 0.04338202719589964 0.13317775041869903 0.010002475244614173 nan [5482.861149544504, 5540.8636415751225, 5535.860296676785] [0.04261585764700658, 0.04393719610940024, 0.0435930278312921] [0.13319770823309843, 0.13299262919296162, 0.13334291383003705] [0.01001077671358305, 0.009972224895283092, 0.010024424124976375] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 300), ('feature_fraction', 0.75), ('min_data_in_leaf', 10), ('lambda_l1', 0.0), ('lambda_l2', 0.0)) 5518.861455168019 0.04379487664828141 0.13332081816428012 0.010012940104660016 nan [5503.861305040489, 5531.863247976415, 5520.859812487156] [0.04296846139124591, 0.04450564422307773, 0.043910524330520596] [0.13314853248502492, 0.13325076664205351, 0.133563155365762] [0.010005087089672711, 0.009991142478415609, 0.01004259074589173] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 200), ('feature_fraction', 0.75), ('min_data_in_leaf', 5), ('lambda_l1', 0.01), ('lambda_l2', 0.0)) 5516.528186339753 0.043428420634351796 0.13328940085375943 0.01001047316856051 nan [5487.860943439267, 5518.8634034527195, 5542.86021212727] [0.04272521338000659, 0.04404691147538778, 0.04351313704766101] [0.1333722297575215, 0.13314960515579993, 0.13334636764795682] [0.010022066308163517, 0.009984104490336437, 0.010025248707181572] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 255), ('feature_fraction', 0.75), ('min_data_in_leaf', 5), ('lambda_l1', 0.01), ('lambda_l2', 0.01)) 5516.195066093779 0.04358077505248376 0.1331689478584154 0.010002020186691402 nan [5498.86128333262, 5526.863622649356, 5522.86029229936] [0.04279281044079247, 0.04419084798604313, 0.04375866673061569] [0.13312767658655858, 0.1330034294197811, 0.13337573756890647] [0.010004087596475593, 0.00997426201144659, 0.010027710952152023] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.01), ('lambda_l2', 0.1)) 5514.5292940883255 0.04264947789468701 0.13228607939237894 0.009937034619459596 nan [5491.8622572435115, 5506.864468447017, 5544.861156574446] [0.04200369085090828, 0.04322038610253584, 0.04272435673061691] [0.13217826934567015, 0.13224586578732075, 0.13243410304414593] [0.009934982574840958, 0.009917588642306645, 0.009958532641231182] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 255), ('feature_fraction', 0.75), ('min_data_in_leaf', 20), ('lambda_l1', 0.01), ('lambda_l2', 0.1)) 5514.194477843622 0.043608389919067285 0.13349542346935464 0.010025744320612234 nan [5492.860902490326, 5518.862951336849, 5530.8595797036905] [0.04281657420404414, 0.044329882537727426, 0.04367871301543028] [0.13347322108081902, 0.13339346377472494, 0.1336195855525199] [0.01002850316366585, 0.010001215498959536, 0.010047514299211314] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 300), ('feature_fraction', 0.75), ('min_data_in_leaf', 20), ('lambda_l1', 0.0), ('lambda_l2', 0.01)) 5513.194455032574 0.0436797875245978 0.13352431544111779 0.010028710453217125 nan [5480.861055602204, 5535.862922100253, 5522.859387395264] [0.04275341588203364, 0.044465800829722804, 0.043820145862036954] [0.1333647458205053, 0.1335040594253311, 0.1337041410775169] [0.010019540207577794, 0.010011607592287405, 0.010054983559786172] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 300), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.01), ('lambda_l2', 0.0)) 5512.195778378197 0.042518762777791846 0.13239644847493523 0.009945033779952924 nan [5475.861958934185, 5516.864199634439, 5543.861176565965] [0.04188413274372922, 0.04318099924785194, 0.042491156341794366] [0.1324711660019806, 0.13239195277759508, 0.13232622664523003] [0.009955818127797809, 0.009927569559774477, 0.009951713652286486] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 255), ('feature_fraction', 0.75), ('min_data_in_leaf', 5), ('lambda_l1', 0.0), ('lambda_l2', 0.0)) 5511.528425436212 0.04353118652313475 0.1331190225162034 0.009998320074606227 nan [5486.861351207182, 5509.863620470923, 5537.8603046305325] [0.0426721825162636, 0.044222779223451435, 0.04369859782968921] [0.1330137392815864, 0.13307286718184305, 0.1332704610851807] [0.00999470592334541, 0.00997998270449768, 0.010020271595975593] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 300), ('feature_fraction', 0.75), ('min_data_in_leaf', 10), ('lambda_l1', 0.01), ('lambda_l2', 0.0)) 5511.528064655493 0.04381337548941063 0.13340733975494723 0.01002017400458448 nan [5482.861212272113, 5526.86315945354, 5524.859822240826] [0.04294998848212514, 0.0445389161636229, 0.04395122182248383] [0.13324637491068944, 0.13340296182022804, 0.13357268253392418] [0.010013054013709314, 0.010002858979206821, 0.010044609020837304] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 300), ('feature_fraction', 0.75), ('min_data_in_leaf', 20), ('lambda_l1', 0.0), ('lambda_l2', 0.1)) 5509.861114902294 0.04369872053738947 0.13347881154289634 0.010025967583497941 nan [5484.860847228678, 5512.862937287781, 5531.859560190418] [0.042815391450454265, 0.04453724194859414, 0.04374352821312001] [0.13347659839878795, 0.13337077237436976, 0.13358906385553132] [0.010029308835216952, 0.010002381643021576, 0.010046212272255295] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 5), ('lambda_l1', 0.1), ('lambda_l2', 0.1)) 5509.529182480059 0.042677454028271535 0.13235869680099602 0.00994230230265753 nan [5466.862073531004, 5524.864255580611, 5536.86121832856] [0.041979618865151634, 0.04341509640855969, 0.04263764681110328] [0.13227417899172675, 0.13244854158976552, 0.13235336982149576] [0.009941333648815099, 0.009932135051834165, 0.00995343820732332] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 200), ('feature_fraction', 0.75), ('min_data_in_leaf', 10), ('lambda_l1', 0.1), ('lambda_l2', 0.1)) 5507.861492480396 0.043329730184335495 0.1332148115720022 0.010006206299551573 nan [5488.861193867805, 5517.863413506121, 5516.859870067261] [0.04258073023124911, 0.043900272062190056, 0.043508188259567315] [0.13315141020410784, 0.13310604073375334, 0.13338698377814542] [0.01000773201509395, 0.009980049548457381, 0.010030837335103387] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 255), ('feature_fraction', 0.75), ('min_data_in_leaf', 10), ('lambda_l1', 0.1), ('lambda_l2', 0.1)) 5506.861462257654 0.043573947329288716 0.133289054254231 0.010011219915078957 nan [5472.861290548025, 5526.863224634684, 5520.859871590257] [0.042837533762747446, 0.04428664946895451, 0.04359765875616419] [0.13315765512480024, 0.13327785828758315, 0.13343164935030963] [0.01000583254220061, 0.009993599939711467, 0.010034227263324792] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 200), ('feature_fraction', 0.5), ('min_data_in_leaf', 20), ('lambda_l1', 0.0), ('lambda_l2', 0.01)) 5506.1955830303095 0.04236379565406328 0.1325028802973507 0.009953664996501544 nan [5483.8617799712565, 5516.8639421709295, 5517.861026948743] [0.04165910502447717, 0.043050527241141445, 0.04238175469657122] [0.1326111680925817, 0.13247857372004326, 0.13241889907942714] [0.009966097002890428, 0.009935011184543131, 0.009959886802071074] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 255), ('feature_fraction', 0.75), ('min_data_in_leaf', 20), ('lambda_l1', 0.1), ('lambda_l2', 0.1)) 5505.861194244175 0.04353016174975993 0.13344817592695993 0.010022666180583102 nan [5484.861033311377, 5501.862947678364, 5530.859601742785] [0.042616736560707524, 0.044315527764192, 0.04365822092438026] [0.13328318845845352, 0.13343416907276562, 0.13362717024966062] [0.010014219758903141, 0.010004687865082636, 0.01004909091776353] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 10), ('lambda_l1', 0.1), ('lambda_l2', 0.0)) 5504.195746061643 0.04256375788989297 0.13245227480099275 0.009948663760269452 nan [5465.861982936095, 5530.864190330706, 5515.861064918127] [0.04186045414517785, 0.04325539742759724, 0.042575422096903806] [0.13243741091861932, 0.13244465565825994, 0.13247475782609894] [0.009952793846551443, 0.00993068521224155, 0.00996251222201536] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 200), ('feature_fraction', 0.75), ('min_data_in_leaf', 20), ('lambda_l1', 0.0), ('lambda_l2', 0.0)) 5501.527700031108 0.04346179850728127 0.13355380007331438 0.010030671125069234 nan [5468.8608421338595, 5508.862806645664, 5526.8594513138] [0.042556963251018015, 0.04419795474384172, 0.04363047752698407] [0.13341903649753337, 0.1336243134491327, 0.13361805027327706] [0.010025747678905787, 0.010017968798490817, 0.010048296897811098] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 200), ('feature_fraction', 0.75), ('min_data_in_leaf', 20), ('lambda_l1', 0.1), ('lambda_l2', 0.01)) 5500.194519504392 0.04338182080196376 0.13348613587032288 0.010025656805221305 nan [5470.860902826693, 5506.86288871286, 5522.859766973624] [0.04253038928526347, 0.04410113932394602, 0.0435139337966818] [0.13347007568067337, 0.13347147571933632, 0.13351685621095893] [0.01002998245449076, 0.010007536355589527, 0.010039451605583624] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 255), ('feature_fraction', 0.5), ('min_data_in_leaf', 20), ('lambda_l1', 0.1), ('lambda_l2', 0.01)) 5495.195511510221 0.042433580119284664 0.13255455432878235 0.009956972714924189 nan [5462.8616010566675, 5505.8638965540495, 5516.861036919944] [0.0417887672978944, 0.043134699886558915, 0.042377273173400676] [0.1327151393616502, 0.1325681378186696, 0.13238038580602723] [0.009974248830038761, 0.009940987146988461, 0.00995568216774534] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 255), ('feature_fraction', 0.75), ('min_data_in_leaf', 20), ('lambda_l1', 0.0), ('lambda_l2', 0.01)) 5494.194307376324 0.043627453539381016 0.1335847724981859 0.010033741093853034 nan [5470.860802919321, 5482.862725309734, 5528.8593938999165] [0.0427007188459237, 0.04441755657609646, 0.04376408519612288] [0.13346973455473732, 0.1335882576247212, 0.1336963253150992] [0.010028516252269642, 0.010017875971260085, 0.010054831058029375] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 255), ('feature_fraction', 0.75), ('min_data_in_leaf', 5), ('lambda_l1', 0.0), ('lambda_l2', 0.1)) 5485.527719506475 0.04352228768679276 0.1337947166760112 0.01004815331369544/home/skyonyttv/.local/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator KMeans from version 0.20.3 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/skyonyttv/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/skyonyttv/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 nan [5457.860598044394, 5505.862739700146, 5492.859820774885] [0.04263431269906107, 0.04423043888359639, 0.04370211147772085] [0.13360064926262513, 0.1340001158496259, 0.13378338491578257] [0.010039872378500866, 0.010046745536554168, 0.010057842026031287] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 8), ('num_leaves', 300), ('feature_fraction', 0.75), ('min_data_in_leaf', 5), ('lambda_l1', 0.01), ('lambda_l2', 0.0)) 5477.527639994735 0.04355268370786425 0.1338881287103237 0.010054501637788005 nan [5456.860564035116, 5493.862690292394, 5481.859665656697] [0.04278728674507745, 0.04429183997763621, 0.04357892440087907] [0.133735414759452, 0.13401539658470596, 0.13391357478681315] [0.010049920089810998, 0.010046140245246818, 0.010067444578306202] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 300), ('feature_fraction', 1.0), ('min_data_in_leaf', 5), ('lambda_l1', 0.01), ('lambda_l2', 0.1)) 5470.858679532171 0.04445666633574596 0.13539364461335437 0.010172701763055288 nan [5470.858679532171] [0.04445666633574596] [0.13539364461335437] [0.010172701763055288] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 255), ('feature_fraction', 1.0), ('min_data_in_leaf', 10), ('lambda_l1', 0.01), ('lambda_l2', 0.1)) 5464.859036916116 0.0440650604935867 0.13508877637426292 0.010150283163450576 nan [5464.859036916116] [0.0440650604935867] [0.13508877637426292] [0.010150283163450576] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 24), ('num_leaves', 300), ('feature_fraction', 1.0), ('min_data_in_leaf', 5), ('lambda_l1', 0.0), ('lambda_l2', 0.0)) 5462.858986315457 0.04527007370889361 0.1357243847951921 0.010191944304737462 nan [5451.858945398395, 5475.8607983875645, 5460.857215160412] [0.044433153287529295, 0.04592025154895246, 0.045456816290199084] [0.13522930348736725, 0.13566747429272186, 0.13627637660548717] [0.01015973057036289, 0.01017249627512116, 0.010243606068728338] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 300), ('feature_fraction', 1.0), ('min_data_in_leaf', 10), ('lambda_l1', 0.1), ('lambda_l2', 0.1)) 5458.85856090192 0.04428724010012428 0.1354906927794073 0.010181440680459525 nan [5458.85856090192] [0.04428724010012428] [0.1354906927794073] [0.010181440680459525] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', 16), ('num_leaves', 300), ('feature_fraction', 1.0), ('min_data_in_leaf', 5), ('lambda_l1', 0.1), ('lambda_l2', 0.0)) 5457.859781412393 0.045066872139312326 0.13556647053001913 0.010174325734863144 nan [5453.858642810601, 5461.860920014186] [0.044430189757168684, 0.04570355452145597] [0.13552500676250248, 0.13560793429753581] [0.010181171359392246, 0.01016748011033404] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 255), ('feature_fraction', 1.0), ('min_data_in_leaf', 10), ('lambda_l1', 0.01), ('lambda_l2', 0.1)) 5456.859076305458 0.04402950555662851 0.1350454648164872 0.010146859151257232 nan [5456.859076305458] [0.04402950555662851] [0.1350454648164872] [0.010146859151257232] []
(('task', 'train'), ('boosting_type', 'gbdt'), ('objective', 'mse'), ('metric', 'mape'), ('learning_rate', 0.01), ('max_depth', -1), ('num_leaves', 255), ('feature_fraction', 1.0), ('min_data_in_leaf', 10), ('lambda_l1', 0.1), ('lambda_l2', 0.0)) 5454.8589905729905 0.044975363927481436 0.13558387229316723 0.010181687254835746 nan [5450.8589620480825, 5456.860792455413, 5456.857217215476] [0.04412943884989719, 0.0457268386127421, 0.045069814319805] [0.13508574644442206, 0.13550640399939665, 0.13615946643568305] [0.010149019369037443, 0.010160262297099851, 0.010235780098369946] []
